介紹：
利用mnist的手寫數字辨識圖片訓練模型
選擇6萬張圖片，每張圖片大小為28x28
Activation Function 為 Sigmoid
Loss Function 為 MSE
隱藏層1層，含有100個神經元

In [2]:
import numpy as np
import neuralnet as nl 
import load_mnist as lm
dataset = lm.load_mnist()
dataset

{'x_train': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 'y_train': array([[0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.]]),
 'x_test': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 'y_test': array([[0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0.,

In [3]:
dataset['x_train'][0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [4]:
dataset['y_train'][0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])

In [5]:
def make_params(shape_list):
    w_list=[]
    b_list=[]
    for i in range(len(shape_list)-1):

        weight=np.random.rand(shape_list[i],shape_list[i+1])
        bias=np.ones(shape_list[i+1])/10.0
        w_list.append(weight)
        b_list.append(bias)
    return w_list,b_list
#第一次先隨機產生權重與設定0.1為偏誤

def sigmoid(x):
    return 1/(1+np.exp(-x))

def inner_product(x_train,w,b):
    return np.dot(x_train,w)+b

def activation(x_train,w,b):
    return sigmoid(inner_product(x_train,w,b))
#啟動函數

def calculate(x_train,w_list,b_list):
    val_dict={}

    a_1=inner_product(x_train,w_list[0],b_list[0])
    y_1=sigmoid(a_1)
    #一次處理的結果

    a_2=inner_product(y_1,w_list[1],b_list[1])
    y_2=sigmoid(a_2)
    #二次處理的結果

    y_2 /=np.sum(y_2,axis=1,keepdims=True) #正規化

    val_dict['y_1']=y_1
    val_dict['y_2']=y_2

    return val_dict

def update(x_train,w_list,b_list,y_train,eta):
    
    val_dict={}
    val_dict=calculate(x_train,w_list,b_list)
    y_1=val_dict['y_1']
    y_2=val_dict['y_2']

    d12_d11=1.0
    d11_d9=1/x_train.shape[0]*(y_2-y_train)
    d9_d8=y_2*(1.0-y_2)
    d8_d7=1.0
    d8_d6=np.transpose(y_1)
    d8_d5=np.transpose(w_list[1])
    d5_d4=y_1*(1-y_1)
    d4_d3=1.0
    d4_d2=np.transpose(x_train)
    
        #以下計算 d12_d7_&_d12_d6
    d12_d8 = d12_d11 * d11_d9 * d9_d8

    b_list[1] -= eta*np.sum(d12_d8 * d8_d7, axis=0)
    w_list[1] -= eta*np.dot(d8_d6, d12_d8)


    #以下計算 d12_d3_&_d12_d2
    d12_d8 = d12_d11 * d11_d9 * d9_d8
    d12_d5 = np.dot(d12_d8, d8_d5)
    d12_d4 = d12_d5 * d5_d4
        
    b_list[0] -= eta * np.sum(d12_d4 * d4_d3, axis=0)
    w_list[0] -= eta * np.dot(d4_d2, d12_d4)

    return w_list, b_list

In [6]:
x_train=dataset['x_train']
y_train=dataset['y_train']
x_test=dataset['x_test']
y_test=dataset['y_test']
w_list,b_list=nl.make_params([784,100,10])

In [8]:
y_test[0:10]

array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [10]:
val_dict=nl.calculate(x_test,w_list,b_list)
val_dict['y_2'][0:10].round(2)


array([[0.19, 0.  , 0.  , 0.  , 0.23, 0.22, 0.  , 0.13, 0.  , 0.23],
       [0.03, 0.  , 0.  , 0.  , 0.41, 0.  , 0.01, 0.04, 0.  , 0.51],
       [0.02, 0.  , 0.  , 0.  , 0.3 , 0.  , 0.  , 0.32, 0.  , 0.35],
       [0.02, 0.  , 0.01, 0.  , 0.3 , 0.  , 0.02, 0.32, 0.  , 0.32],
       [0.01, 0.  , 0.07, 0.  , 0.28, 0.21, 0.02, 0.13, 0.  , 0.28],
       [0.01, 0.  , 0.  , 0.  , 0.02, 0.  , 0.  , 0.42, 0.  , 0.55],
       [0.  , 0.  , 0.29, 0.  , 0.3 , 0.01, 0.  , 0.09, 0.  , 0.3 ],
       [0.11, 0.03, 0.01, 0.  , 0.4 , 0.  , 0.  , 0.02, 0.01, 0.41],
       [0.  , 0.  , 0.  , 0.  , 0.44, 0.  , 0.  , 0.11, 0.  , 0.44],
       [0.  , 0.  , 0.  , 0.  , 0.3 , 0.03, 0.07, 0.29, 0.  , 0.3 ]])

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(dataset['x_test'][8].reshape(28,28),cmap="gray")
plt.axis("off")
plt.show()


In [12]:
for epoch in range(100):
    ra=np.random.randint(60000,size=60000) #建立亂數的索引
    for i in range(60): #60批次
        x_batch=x_train[ra[i*1000:(i+1)*1000],:]
        y_batch=y_train[ra[i*1000:(i+1)*1000],:]
        w_list,b_list=nl.update(x_batch,w_list,b_list,y_batch,eta=0.2)

In [ ]:
val_dict=nl.calculate(x_test,w_list,b_list)
val_dict['y_2'][0:10].round(2)